# Final Project Hoang Nam Huynh-Viet

## 1. Install the neccecery libraries and packages


In [ ]:
!pip install datasets
!pip install --no-cache-dir transformers
!pip install transformers[torch]
!pip install bertviz
!pip install bertviz transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
from datasets import load_dataset, DatasetDict, Dataset
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AlbertTokenizer,  AlbertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from torch.utils.tensorboard import SummaryWriter
from bertviz import head_view, model_view
from bertviz.neuron_view import show
import torch
import os

## 2. Load the models and dataset

In [ ]:
amazon_dataset = load_dataset('jhan21/amazon-food-reviews-dataset')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
amazon_dataset = amazon_dataset.remove_columns(['HelpfulnessNumerator', 'HelpfulnessDenominator', 'UserId', 'Id', 'ProductId', 'ProfileName', 'Time', 'Summary'])
print(amazon_dataset)

DatasetDict({
    train: Dataset({
        features: ['Score', 'Text'],
        num_rows: 568454
    })
})


In [ ]:
distilbert_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
albert_tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")
distilbert_model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=5)
albert_model = AlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=5)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def truncate(example):
  return{
      'Text': " ".join(example['Text'].split()[:50]),
      'Score': example['Score'] - 1
  }
small_amazon_dataset = DatasetDict(
    train = amazon_dataset['train'].shuffle(seed=24).select(range(250)).map(truncate),
    val = amazon_dataset['train'].shuffle(seed=24).select(range(250,275)).map(truncate),
    test = amazon_dataset['train'].shuffle(seed=24).select(range(275, 300)).map(truncate),
)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

## 3. Tokenization

In [ ]:
def distilbert_tokenize_function(examples):
    return distilbert_tokenizer(examples['Text'], padding=True, truncation=True)
viz_distilbert_small_tokenized_dataset = small_amazon_dataset.map(distilbert_tokenize_function, batched = True, batch_size = 16)
viz_distilbert_small_tokenized_dataset = viz_distilbert_small_tokenized_dataset.rename_column('Score', 'labels')
distilbert_small_tokenized_dataset = small_amazon_dataset.map(distilbert_tokenize_function, batched = True, batch_size = 16)
distilbert_small_tokenized_dataset = distilbert_small_tokenized_dataset.remove_columns(['Text'])
distilbert_small_tokenized_dataset = distilbert_small_tokenized_dataset.rename_column('Score', 'labels')

print(distilbert_small_tokenized_dataset)

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 250
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 25
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 25
    })
})


In [ ]:
def albert_tokenize_function(examples):
    return albert_tokenizer(examples['Text'], padding=True, truncation=True)
viz_albert_small_tokenized_dataset = small_amazon_dataset.map(albert_tokenize_function, batched = True, batch_size = 16)
viz_albert_small_tokenized_dataset = viz_albert_small_tokenized_dataset.rename_column('Score', 'labels')
albert_small_tokenized_dataset = small_amazon_dataset.map(albert_tokenize_function, batched = True, batch_size = 16)
albert_small_tokenized_dataset = albert_small_tokenized_dataset.remove_columns(['Text'])
albert_small_tokenized_dataset = albert_small_tokenized_dataset.rename_column('Score', 'labels')

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Map:   0%|          | 0/25 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

## 4. Training arguments and Finetuning

### Compute metrics

In [ ]:
def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = np.mean(predictions == labels)
    precision = precision_score(labels, predictions, average='weighted', zero_division=1)
    recall = recall_score(labels, predictions, average='weighted', zero_division=1)
    f1 = f1_score(labels, predictions, average='weighted', zero_division=1)

    metrics = {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

    return metrics

### Distilbert

In [ ]:
distilbert_arguments = TrainingArguments(
    output_dir="sample_distilbert_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    weight_decay=0.01,
    seed=224
)

trainer_for_distilbert = Trainer(
    model=distilbert_model,
    args=distilbert_arguments,
    train_dataset=distilbert_small_tokenized_dataset['train'],
    eval_dataset=distilbert_small_tokenized_dataset['val'],
    tokenizer=distilbert_tokenizer,
    compute_metrics=compute_metrics
)

### Albert

In [ ]:
albert_arguments = TrainingArguments(
    output_dir="sample_albert_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    weight_decay=0.01,
    seed=224
)

trainer_for_albert = Trainer(
    model=albert_model,
    args=albert_arguments,
    train_dataset=albert_small_tokenized_dataset['train'],
    eval_dataset=albert_small_tokenized_dataset['val'],
    tokenizer=albert_tokenizer,
    compute_metrics=compute_metrics
)

## 5. Train and Result

### Train distilbert

In [ ]:
trainer_for_distilbert.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.079667,0.680000,0.782400,0.680000,0.550476
2,No log,0.990562,0.680000,0.782400,0.680000,0.550476
3,No log,0.965167,0.680000,0.782400,0.680000,0.550476
4,No log,0.950717,0.680000,0.782400,0.680000,0.550476
5,No log,0.993596,0.680000,0.782400,0.680000,0.550476
6,No log,0.894753,0.680000,0.782400,0.680000,0.550476
7,No log,0.897667,0.640000,0.565714,0.640000,0.576842
8,No log,0.899570,0.680000,0.553043,0.680000,0.584000
9,No log,0.887251,0.680000,0.688889,0.680000,0.664000
10,No log,0.875745,0.680000,0.688889,0.680000,0.664000


TrainOutput(global_step=160, training_loss=0.9893636703491211, metrics={'train_runtime': 1225.0421, 'train_samples_per_second': 2.041, 'train_steps_per_second': 0.131, 'total_flos': 61126108130580.0, 'train_loss': 0.9893636703491211, 'epoch': 10.0})

### Train albert

In [ ]:
trainer_for_albert.train()

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.971230,0.680000,0.782400,0.680000,0.550476
2,No log,0.936845,0.680000,0.782400,0.680000,0.550476
3,No log,0.860011,0.680000,0.782400,0.680000,0.550476
4,No log,0.848585,0.720000,0.840476,0.720000,0.624421
5,No log,0.843825,0.640000,0.584842,0.640000,0.580000
6,No log,0.690596,0.720000,0.704000,0.720000,0.681441
7,No log,0.723432,0.720000,0.752632,0.720000,0.691111
8,No log,0.676681,0.720000,0.717333,0.720000,0.668108
9,No log,0.661385,0.720000,0.744000,0.720000,0.688108
10,No log,0.672633,0.720000,0.724000,0.720000,0.674775


TrainOutput(global_step=160, training_loss=0.6439356327056884, metrics={'train_runtime': 2392.7101, 'train_samples_per_second': 1.045, 'train_steps_per_second': 0.067, 'total_flos': 11817380707488.0, 'train_loss': 0.6439356327056884, 'epoch': 10.0})

In [ ]:
distilbert_results = trainer_for_distilbert.predict(distilbert_small_tokenized_dataset['test'])
print(distilbert_results)

albert_results = trainer_for_albert.predict(albert_small_tokenized_dataset['test'])
print(albert_results)

PredictionOutput(predictions=array([[-0.8811746 , -1.2164462 , -0.73144114, -0.31288975,  2.8092852 ],
       [-0.3266972 , -0.5849592 , -0.2815924 ,  0.47634628,  0.31175867],
       [ 0.20120448, -0.63149005, -0.29395008,  0.26304355,  0.09032939],
       [-0.8276712 , -1.0403216 , -0.55688   ,  0.44500107,  1.675994  ],
       [-0.684454  , -1.158956  , -0.46804845,  0.33967966,  1.5683701 ],
       [-0.03328332, -0.2158325 , -0.1432989 ,  0.22570631, -0.3493775 ],
       [-0.30467385, -0.9144177 , -0.5222584 , -0.35079786,  1.5870063 ],
       [-0.9270637 , -1.2158773 , -0.69008106, -0.20562692,  2.7507477 ],
       [-0.9336663 , -1.1272055 , -0.7810847 , -0.15923944,  2.6817193 ],
       [-0.6917006 , -1.0332096 , -0.5315407 ,  0.20268545,  1.6010292 ],
       [ 0.11072154, -0.5914719 , -0.21323143,  0.08078369,  0.31616044],
       [-0.8922166 , -1.109854  , -0.7865493 , -0.1858349 ,  2.6857042 ],
       [-0.13356273, -0.7993532 , -0.23183101,  0.40173742,  0.24714798],
       [-

PredictionOutput(predictions=array([[-1.0936564 , -1.7006264 , -0.90363216, -0.71282035,  3.401814  ],
       [-1.366194  , -1.1057645 , -0.72361434,  1.2963022 ,  1.778676  ],
       [-0.63534486, -0.97437394, -0.53258675,  0.94021535,  0.9069376 ],
       [-1.5819561 , -1.1246897 , -1.009711  ,  0.08386239,  3.384508  ],
       [-1.3831289 , -1.2753786 , -1.1728513 ,  0.45559973,  3.1807508 ],
       [ 0.06717423, -0.6554622 , -0.08671506, -0.05767263, -0.45528987],
       [ 0.8116817 , -0.81867903,  0.30524477, -0.97503364, -0.4252983 ],
       [-1.2862805 , -1.4848845 , -1.0442488 , -0.4675581 ,  3.6296005 ],
       [-1.5252213 , -0.72655296, -1.0465481 ,  0.3536862 ,  3.2717607 ],
       [-0.13151625, -0.54201716,  0.5139975 , -0.609974  , -0.46283367],
       [-1.4139152 , -1.178412  , -1.0695417 , -0.22786576,  3.56915   ],
       [-1.3114921 , -1.3730819 , -1.1515315 , -0.36833745,  3.7420366 ],
       [ 0.31239304, -1.6285048 , -0.6901556 , -0.15917969,  0.41130072],
       [-

In [ ]:
test_sentences = ["This is not good at all", "this is not good", "it is okay", "i like this dish", "this is really good"]

fine_tuned_model_distilbert = AutoModelForSequenceClassification.from_pretrained("sample_albert_trainer/checkpoint-128")

for test_str in test_sentences:
  model_inputs = distilbert_tokenizer(test_str, return_tensors="pt")
  prediction = torch.argmax(fine_tuned_model_distilbert(**model_inputs).logits)
  print(["NEGATIVE", "SOMEWHAT NEGATIVE", "NEUTRAL", "SOMEWHAT POSITIVE", "POSITIVE"][prediction])

SOMEWHAT POSITIVE
POSITIVE
SOMEWHAT POSITIVE
NEUTRAL
SOMEWHAT POSITIVE


In [ ]:
test_sentences = ["This is not good at all", "this is not good", "it is okay", "i like this dish", "this is really good"]

fine_tuned_model_albert = AutoModelForSequenceClassification.from_pretrained("sample_albert_trainer/checkpoint-128")

for test_str in test_sentences:
  model_inputs = albert_tokenizer(test_str, return_tensors="pt")
  prediction = torch.argmax(fine_tuned_model_albert(**model_inputs).logits)
  print(["NEGATIVE", "SOMEWHAT NEGATIVE", "NEUTRAL", "SOMEWHAT POSITIVE", "POSITIVE"][prediction])

POSITIVE
NEGATIVE
POSITIVE
POSITIVE
POSITIVE


## **Visualization part**

## Store the hidden states

In [ ]:
model_inputs_distilbert = distilbert_tokenizer(viz_distilbert_small_tokenized_dataset['test']['Text'], padding=True, truncation=True, return_tensors='pt')
outputs_distilbert = fine_tuned_model_distilbert(**model_inputs_distilbert, output_hidden_states=True)

model_inputs_albert = albert_tokenizer(viz_albert_small_tokenized_dataset['test']['Text'], padding=True, truncation=True, return_tensors='pt')
outputs_albert = fine_tuned_model_albert(**model_inputs_albert, output_hidden_states=True)

In [ ]:
path = "/content/drive/MyDrive/results_viz_distilbert"
layer=0
if not os.path.exists(path):
  os.mkdir(path)

while layer in range(len(outputs_distilbert['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs_distilbert['hidden_states'][layer])):
    sp_token_position = 0
    for token in model_inputs_distilbert['input_ids'][example]:
      if token != 101:
        sp_token_position += 1
      else:
        tensor = outputs_distilbert['hidden_states'][layer][example][sp_token_position]
        tensors.append(tensor)
        break

    label = [viz_distilbert_small_tokenized_dataset['test']['Text'][example], str(viz_distilbert_small_tokenized_dataset['test']['labels'][example])]
    labels.append(label)
    example +=1

  writer = SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['Text', 'Emotion'])

  layer+=1

In [ ]:
path = "/content/drive/MyDrive/results_viz_albert"
layer=0
if not os.path.exists(path):
  os.mkdir(path)

while layer in range(len(outputs_albert['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs_albert['hidden_states'][layer])):
    sp_token_position = 0
    for token in model_inputs_albert['input_ids'][example]:
      if token != 2:
        sp_token_position += 1
      else:
        tensor = outputs_albert['hidden_states'][layer][example][sp_token_position]
        tensors.append(tensor)
        break

    label = [viz_albert_small_tokenized_dataset['test']['Text'][example], str(viz_albert_small_tokenized_dataset['test']['labels'][example])]
    labels.append(label)
    example +=1

  writer = SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['Text', 'Emotion'])

  layer+=1